## sqlite3 内存模式 & 持久化

目的：在python下使用sqlite3的**内存模式**，并将需要的结果进行持久化的保存，在下一次需要的时候重新加载到内存中。

解决方法：查阅了sqlite3的相关文档后，发现python中并未提供现有的api可供调用。
不过发现了iterdump()方法，其描述是"Returns an iterator to dump the database in an SQL text format. Useful when saving an in-memory database for later restoration." <br />
因此我们的思路是在使用时将文件库导入到内存库中使用，当使用结束时再将内存库导出到文件库中以实现持久化的存储。

In [66]:
import os
import sqlite3
from io import StringIO

实现：
* init_sqlite_db_from_disk(): 将硬盘文件库加载到内存中
* save_memory_db_to_disk(): 将内存数据库导出到硬盘中

In [192]:
def init_sqlite_db_from_disk(db_path, extension=".db"):
    con = sqlite3.connect(db_path+extension)
    tempfile = StringIO()
    for line in con.iterdump():
        tempfile.write('%s\n' % line)
    con.close()
    tempfile.seek(0)

    memory_db = sqlite3.connect(":memory:")
    memory_db.cursor().executescript(tempfile.read())
    memory_db.commit()
    
    return memory_db

def save_memory_db_to_disk(db, db_path, extension=".db"):
    tempfile = StringIO()
    for line in db.iterdump():
        tempfile.write('%s\n' % line)
    tempfile.seek(0)
    
    if(os.path.exists(db_path+extension)):
        os.remove(db_path+extension) # 删除原来的记录
        
    disk_db = sqlite3.connect(db_path+extension)
    # print(tempfile.read())
    disk_db.cursor().executescript(tempfile.read())
    disk_db.commit()
    disk_db.close()

code example:

In [84]:
conn = sqlite3.connect(":memory:")

c = conn.cursor()
c.execute('create table user (id varchar(20) primary key, name varchar(20))')
c.execute('insert into user (id, name) values ("1", "Michael")')

conn.commit()

In [93]:
for row in conn.execute('select * from user'):
    print(row)

('1', 'Michael')


In [115]:
save_memory_db_to_disk(conn, "test1")

In [116]:
test = sqlite3.connect("test1.db")
cur = test.cursor()

for row in cur.execute('select * from user'):
    print(row)

('1', 'Michael')


sqlite3 内存模式与文件模式性能比较：

In [190]:
import time 

count = 10000

# 插入
def test_insert(db):
    start = time.time()
    c = db.cursor()
    nums = list(range(count))
    nums = [(i,) for i in nums]
    c.executemany('insert into test values(?)', nums)
    c.close()
    db.commit()
    
    print(str(time.time() - start) + ' insert')
    
# 查询
def test_query(db):
    start = time.time()
    c = db.cursor()
    for i in range(count):
        c.execute('select * from test where num=?', (i,))
    c.close()
    print(str(time.time() - start) + ' query')

In [172]:
memory = sqlite3.connect(":memory:")
test = sqlite3.connect("test1.db")

In [184]:
memory.cursor().execute('drop table test')
test.cursor().execute('drop table test')
memory.cursor().execute('create table test (num integer)')
test.cursor().execute('create table test (num integer)')

In [186]:
test_insert(memory)
test_insert(test)

10.144698858261108 insert
9.35562014579773 insert


In [191]:
# 10000条数据
test_query(memory)
test_query(test)

4.39375901222229 query
4.500895023345947 query


内存数据库和文件数据库并没有表现出很大的性能差异，原因？